## **DATA AVAILABILITY NOTICE**
##### This script requires external datasets that are not included in this repository due to copyright or privacy restrictions.
##### Please refer to the citations in the manuscript to download the data or apply for access permissions.

# Descriptive statistics

In [ ]:
import pandas as pd

def print_grouped_stats(df: pd.DataFrame, group_col: str, target_col: str):
    stats = df.groupby(group_col)[target_col].agg(['mean', 'std', 'min', 'max', 'count'])

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.float_format', '{:.4f}'.format) 

    print("-" * 60)
    print(stats)
    print("-" * 60)

df = pd.read_excel('norms_gpt4o_character_prompt.xlsx') #norms_gpt4o_word_prompt.xlsx, norms_gpt4o_expression_prompt.xlsx
print_grouped_stats(df, group_col='Length', target_col='GPT_FAM_probs')

In [ ]:
import pandas as pd

def print_grouped_stats(df: pd.DataFrame, group_col: str, target_col: str):
    stats = df.groupby(group_col)[target_col].agg(['mean', 'std', 'min', 'max', 'count'])

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.float_format', '{:.4f}'.format) 

    print("-" * 60)
    print(stats)
    print("-" * 60)

df = pd.read_excel('norms_qwen_max.xlsx')
print_grouped_stats(df, group_col='Length', target_col='qwen_FAM_mean_30')

In [ ]:
# paired samples t-test
import pandas as pd
from scipy import stats

def perform_paired_ttest(word_file: str, expr_file: str):

    df_word = pd.read_excel(word_file)
    df_expr = pd.read_excel(expr_file)

    df_word_clean = df_word[['WORD', 'Length', 'GPT_FAM_probs']]
    df_expr_clean = df_expr[['WORD', 'GPT_FAM_probs']].rename(
        columns={'GPT_FAM_probs': 'GPT_FAM_probs_expr'}
    )

    merged_df = pd.merge(df_word_clean, df_expr_clean, on='WORD')

    print("\n>>> Paired Samples t-test Results by Length Group")
    print("=" * 65)

    for length, group in merged_df.groupby('Length'):
        
        n = len(group)
        df = n - 1
        
        t_stat, p_val = stats.ttest_rel(group['GPT_FAM_probs'], group['GPT_FAM_probs_expr'])
        
        diff = group['GPT_FAM_probs'] - group['GPT_FAM_probs_expr']
        cohen_d = diff.mean() / diff.std(ddof=1)

        mean_word, std_word = group['GPT_FAM_probs'].mean(), group['GPT_FAM_probs'].std()
        mean_expr, std_expr = group['GPT_FAM_probs_expr'].mean(), group['GPT_FAM_probs_expr'].std()

        print(f"Group Length: {length} (N={n})")
        print("-" * 30)
        print(f"Stats:  t = {t_stat:.4f} | p = {p_val:.4f} | df = {df}")
        print(f"Effect: Cohen's d = {cohen_d:.4f}")
        print(f"Word:   Mean = {mean_word:.4f}, Std = {std_word:.4f}")
        print(f"Expr:   Mean = {mean_expr:.4f}, Std = {std_expr:.4f}")
        print("=" * 65)

WORD_FILE = 'norms_gpt4o_word_prompt.xlsx'
EXPR_FILE = 'norms_gpt4o_expression_prompt.xlsx'

perform_paired_ttest(WORD_FILE, EXPR_FILE)

# Correlation

In [ ]:
# Scatter plots of GPT-4o versus human familiarity ratings across word lengths
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from scipy.stats import pearsonr

# ==========================================
# 1. CONFIGURATION
# ==========================================
# File Paths 
PATH_CHAR = r"D:\0 ECNU\CAILAB\LLM_Familiarity\Code\norms_gpt4o_expression_prompt.xlsx" 
PATH_WORD = r"D:\0 ECNU\CAILAB\LLM_familiarity\Code\norms_gpt4o_word_prompt.xlsx" 
PATH_EXPR = r"D:\0 ECNU\CAILAB\LLM_familiarity\Code\norms_gpt4o_expression_prompt.xlsx"

# Visual Settings
FONT_FAMILY = 'Arial' 
plt.rcParams['font.family'] = FONT_FAMILY
plt.rcParams['font.size'] = 12

# Plotting Parameters
AXIS_LIMITS = [0.8, 7.2]   # Extended range to prevent clipping
TICKS = range(1, 8)        # Ticks: 1 to 7
DOT_SIZE = 10              # Scatter dot size
ALPHA = 0.7                # Transparency

# ==========================================
# 2. DATA LOADING & PREPROCESSING
# ==========================================
def load_and_filter(path, length_val=None, x_col='', y_col='', c_col='SUBTLEX_logWF'):
    """
    Loads dataset, filters by length, and selects relevant columns.
    """
    df = pd.read_excel(path)

    # Filter by Length if specified
    if length_val:
        df = df[df['Length'] == length_val]
    
    # Ensure numeric types
    cols = [x_col, y_col, c_col]
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    
    return df[cols].dropna()

# Load Datasets
print("Loading datasets...")
df_A = load_and_filter(PATH_CHAR, 1, 'Human_FAM_Liu', 'GPT_FAM_probs')
df_B = load_and_filter(PATH_WORD, 2, 'Human_FAM_Su', 'GPT_FAM_probs')
df_C = load_and_filter(PATH_WORD, 3, 'Human_FAM_Su', 'GPT_FAM_probs')
df_D = load_and_filter(PATH_EXPR, 4, 'Human_FAM_Su', 'GPT_FAM_probs')

# global normalization for color mapping (Word Frequency)
all_wf = pd.concat([df_A['SUBTLEX_logWF'], df_B['SUBTLEX_logWF'], 
                    df_C['SUBTLEX_logWF'], df_D['SUBTLEX_logWF']])
norm = mcolors.Normalize(vmin=all_wf.min(), vmax=all_wf.max()) 

# ==========================================
# 3. PLOTTING LOGIC
# ==========================================
# Initialize canvas
fig, axes = plt.subplots(1, 4, figsize=(18, 4.5), sharey=True, constrained_layout=True)

def plot_subplot(ax, df, x_col, title_label):
    if df.empty:
        ax.set_title("No Data")
        return None
        
    x = df[x_col]
    y = df['GPT_FAM_probs']
    c = df['SUBTLEX_logWF']
    n = len(df)
    
    # 1. Scatter Plot
    sc = ax.scatter(x, y, c=c, cmap='viridis', norm=norm, 
                    s=DOT_SIZE, alpha=ALPHA, edgecolors='none')
    
    # 2. Regression Line
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    x_range = np.linspace(x.min(), x.max(), 100) 
    ax.plot(x_range, p(x_range), 'k--', lw=1.5)
    
    # 3. Pearson Correlation
    r, p_val = pearsonr(x, y)
    star = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
    ax.text(0.95, 0.05, f"$r$ = {r:.2f}{star}", transform=ax.transAxes, 
            ha='right', va='bottom', fontsize=12, color='#1a3b32')
    
    # 4. Axis Formatting
    ax.set_title(f"{title_label} (N={n:,})", fontsize=14, pad=10)
    ax.set_xlim(AXIS_LIMITS)
    ax.set_ylim(AXIS_LIMITS)
    ax.set_xticks(TICKS)
    ax.set_yticks(TICKS)
    ax.set_box_aspect(1) 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Force display of Y-axis ticks/labels for all subplots (even with sharey=True)
    ax.tick_params(axis='y', labelleft=True, left=True)
    
    return sc

# Generate Subplots
print("Generating plots...")
sc_A = plot_subplot(axes[0], df_A, 'Human_FAM_Liu', 'Word length = 1')
sc_B = plot_subplot(axes[1], df_B, 'Human_FAM_Su', 'Word length = 2')
sc_C = plot_subplot(axes[2], df_C, 'Human_FAM_Su', 'Word length = 3')
sc_D = plot_subplot(axes[3], df_D, 'Human_FAM_Su', 'Word length = 4')

# ==========================================
# 4. FINAL TOUCHES & DISPLAY
# ==========================================
# Global Labels (Adjusted positions as requested)
fig.supxlabel('Human FAM', fontsize=16, y=-0.06)
fig.supylabel('GPT FAM', fontsize=16, x=-0.02) 

# Colorbar (Using the last plot's mappable)
if sc_D:
    cbar = fig.colorbar(sc_D, ax=axes, orientation='vertical', fraction=0.02, pad=0.02)
    cbar.set_label('Word Frequency (Log WF)', fontsize=12, rotation=270, labelpad=20)

print("Displaying plot window...")
plt.show()

In [ ]:
# Compute bivariate correlations by word length - single-character items
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Load dataset
data = pd.read_excel('norms_gpt4o_expression_prompt.xlsx')  

# Group by Length and compute Pearson correlation per group
results = []
for name, group in data.groupby('Length'):
    # Drop rows with NaN or infinite values in the relevant columns
    group = group.replace([np.inf, -np.inf], np.nan).dropna(subset=['GPT_FAM_probs','Human_FAM_Liu']) # 'SUBTLEX_logWF','SUBTLEX_logW_CD','SUBTLEX_logCHR','SUBTLEX_logCHR_CD'(当算和词频的相关的时候)
    
    # Compute sample size
    sample_size = group.shape[0]

    # Require at least 2 observations to compute correlation
    if sample_size > 1:
        # Compute Pearson r and p-value, format to 4 decimals
        r, p_value = pearsonr(group['GPT_FAM_probs'], group['Human_FAM_Liu'])
        results.append((name, sample_size, f"{r:.4f}", f"{p_value:.4f}"))
    else:
        # Not enough data for this length
        results.append((name, sample_size, "N/A", "N/A"))

# Convert results to DataFrame for display
result_df = pd.DataFrame(results, columns=['WordLength', 'SampleSize', 'Pearsonr', 'PValue'])

# Print results
print(result_df)

In [ ]:
# Compute bivariate correlations by word length - multi-character items
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Load data
data = pd.read_excel('norms_gpt4o_word_prompt.xlsx')  

# Group by Length and compute correlation per group
results = []
for name, group in data.groupby('Length'):
    # Remove rows with NaN or infinite values (keep relevant English tokens)
    group = group.replace([np.inf, -np.inf], np.nan).dropna(subset=['GPT_FAM_probs', 'Human_FAM_M_Su']) # 'SUBTLEX_logWF','SUBTLEX_logW_CD'
    
    # Compute sample size
    sample_size = group.shape[0]

    # Require at least 2 observations to compute Pearson r
    if sample_size > 1:
        r, p_value = pearsonr(group['GPT_FAM_probs'], group['Human_FAM_M_Su'])
        # Format r and p-value to 3 decimals
        results.append((name, sample_size, f"{r:.3f}", f"{p_value:.3f}"))
    else:
        # Not enough data for this length
        results.append((name, sample_size, "N/A", "N/A"))

# Convert results to DataFrame for display
result_df = pd.DataFrame(results, columns=['WordLength', 'SampleSize', 'Pearsonr', 'PValue'])

# Print results
print(result_df)

In [ ]:
# Compute partial correlation controlling for Length
import pandas as pd
import pingouin as pg

# Load Excel file into a DataFrame
file_path = 'norms_gpt4o_word_prompt.xlsx' 
df = pd.read_excel(file_path)

# Drop rows missing the variables needed for the analysis
df_cleaned = df.dropna(subset=['GPT_FAM_probs', 'Human_FAM_M_Su', 'Length'])

# Compute partial correlation controlling for Length
partial_corr_result = pg.partial_corr(data=df_cleaned, x='GPT_FAM_probs', y='Human_FAM_M_Su', covar='Length')

# Print the partial correlation result; format p-value to 4 decimal places
print(partial_corr_result.to_string(formatters={'p-val': '{:.4f}'.format}))

# Regression

## Univariate Linear Regression

In [ ]:
# Univariate linear regression - single-character items - LDT task
import pandas as pd
import statsmodels.formula.api as smf

# Load data file and specify columns to read
file_path = '27624_expression_7_cleaned_filtered.xlsx'  # Excel file path
y_column = 'zRT_LDT'  # dependent variable
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_Liu","SUBTLEX_logWF","SUBTLEX_logW_CD","SUBTLEX_logCHR","SUBTLEX_logCHR_CD"]  # predictors
length_column = 'Length'  # word length column

# Read Excel and select needed columns
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Remove rows with missing values in any predictor or the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Collect regression results
final_results = []

# Loop over each Length group and run univariate OLS for each predictor
lengths = data_cleaned[length_column].unique()
for length in lengths:
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    for x in x_columns:
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)
        })

# Convert results list to DataFrame and display
final_results_df = pd.DataFrame(final_results)
print(final_results_df)

In [ ]:
# Univariate linear regression - single-character items - Naming task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_expression_7_cleaned.xlsx'  
y_column = 'zRT_Nam_Liu'  # Dependent variable column name
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_Liu","SUBTLEX_logWF","SUBTLEX_logW_CD","SUBTLEX_logCHR","SUBTLEX_logCHR_CD"]  # Predictor variable names
length_column = 'Length'  # Word length column nam

# Read Excel file (select columns)
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Drop rows with missing values in predictors or outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results for each length
final_results = []

# Iterate over word lengths
lengths = data_cleaned[length_column].unique()  # Get unique lengths
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate regression for each predictor
    for x in x_columns:
        # Fit OLS model
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  # Add sample size
        })

# Convert results to DataFrame
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

In [ ]:
# Univariate linear regression - multi-character words - LDT task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned_filtered.xlsx'  
y_column = 'zRT_LDT' 
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Clean dataset: keep rows with no missing values in predictors and outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results per word length
final_results = []

# Group by word length
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for the current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate regression for each predictor
    for x in x_columns:
        # Fit OLS model for y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  # 添加样本量
        })

# Convert results to DataFrame and display
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

In [ ]:
# Univariate linear regression - multi-character words - Naming task -Zhang et al.,2023
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned_filtered_nam.xlsx'  
y_column = 'zRT_Nam_Zhang'   
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Ensure rows include no missing values for all predictors and the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results
final_results = []

# Iterate over word lengths and run univariate regression for each predictor
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate OLS for each predictor
    for x in x_columns:
        # Fit the model y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  
        })

# Convert results to DataFrame and print
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

In [ ]:
# Univariate linear regression - multi-character words - Naming task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned.xlsx'  
y_column = 'zRT_Nam_Hendrix'  # zRT_Nam_Hendrix 
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Ensure rows include no missing values for all predictors and the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results
final_results = []

# Iterate over word lengths and run univariate regression for each predictor
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate OLS for each predictor
    for x in x_columns:
        # Fit the model y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  
        })

# Convert results to DataFrame and print
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

## Univariate Nonlinear Regression

In [ ]:
# Univariate Nonlinear regression - single-character words - LDT task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_expression_7_cleaned_filtered.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_LDT'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_Liu',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'SUBTLEX_logCHR',
    'SUBTLEX_logCHR_CD'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")

In [ ]:
# Univariate Nonlinear regression - single-character words - Naming task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_expression_7_cleaned.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Liu'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_Liu',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'SUBTLEX_logCHR',
    'SUBTLEX_logCHR_CD'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")

In [ ]:
# Univariate Nonlinear regression - multi-character words - LDT task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned_filtered.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_LDT'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")

In [ ]:
# Univariate Nonlinear regression - multi-character words - Naming task - Zhang et al.,2023

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned_filtered_nam.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Zhang'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")

In [ ]:
# Univariate Nonlinear regression - multi-character words - Naming task - Hendrix et al.,2022

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Hendrix'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")

## Hierarchical Regression

In [ ]:
# Hierarchical regression for LDT (single-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_expression_7_cleaned_filtered.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_LDT"
steps = {
    "Step1": ["NoS", "SUBTLEX_logCHR_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA_Liu"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_Liu_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"]                                  # GPT familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

In [ ]:
# Hierarchical regression for Naming (single-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_expression_7_cleaned.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Liu"
steps = {
    "Step1": ["NoS", "SUBTLEX_logCHR_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA_Liu"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_Liu_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"]                                  # GPT familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

In [ ]:
# # Hierarchical regression for LDT (multi-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned_filtered.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_LDT"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

In [ ]:
# # Hierarchical regression for Naming (multi-character)(Zhang et al.,2023)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned_filtered_nam.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Zhang"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

In [ ]:
# # Hierarchical regression for Naming (multi-character)(Hendrix et al.,2020)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Hendrix"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)